# Final Assignment

# Excercise 1: Implement and query stored procedure:
## Step 1:

Establish a connection 

Import MySQLConnectionPool.

Import Error.

Create a pool named pool_a with two connections. You need to use the try-except block to handle any possible errors. 



In [1]:
import mysql.connector as connector

In [2]:
# Establish connection between Python and MySQL database via MySQL Connector/Python API
connection=connector.connect(
                             user="root",
                             password="",
                            )

In [3]:
# Create cursor object to communicate with entire MySQL database
cursor = connection.cursor()

Once the connection is established and you have a cursor object, you can select the database `little_lemon` using the code below and work with the respective table to complete the required tasks.  

In [4]:
# Set the little_lemon database for use 
cursor.execute("use little_lemon_db")

# Confirm the datbase in use
connection.database

'little_lemon_db'

In [5]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon_db", 
    "user":"root", 
    "password":""
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_a",pool_size = 2,**dbconfig)
    print("The connection pool is created with the pool name: ", pool.pool_name)
    print("The pool size: ", pool.pool_size)

except Error as er:
    print("Error code: ", er.errno)
    print("Error message:", er.msg)

The connection pool is created with the pool name:  pool_a
The pool size:  2


Get a connection from the pool and create the cursor object:

In [6]:
print("Get the coonection from the pool.")
connection1 = pool.get_connection()
print("'connection1' object is created with a connection from the pool")

cursor = connection1.cursor()

Get the coonection from the pool.
'connection1' object is created with a connection from the pool


## Step 2:

Implement a stored procedure called PeakHours.

Write a SQL CREATE PROCEDURE query for PeakHours:

Use HOUR to extract the hour data from the BookingSlot.

Use COUNT on the hour part of the BookingSlot data to count the number of bookings. 

Use GROUP BY on the booking hour.

Use ORDER BY to retrieve the number of bookings in descending order.

Run the stored procedure query by invoking the execute module on the cursor.

Invoke callproc to call the stored procedure.

Fetch the results in a variable called dataset.

Extract the names of the columns.

Print the names of the columns.

Print the sorted data using a for loop.

In [7]:
connection = pool.get_connection()
cursor = connection.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS PeakHours;")

stored_procedure_query = """
CREATE PROCEDURE PeakHours()
BEGIN

SELECT
HOUR(BookingSlot) AS Booking_Hour,
COUNT(HOUR(BookingSlot)) AS n_Booking
FROM Bookings
GROUP BY Booking_Hour
ORDER BY n_Booking DESC;

END
"""
#Execute the query.
cursor.execute(stored_procedure_query)
#Call The procedure
cursor.callproc("PeakHours")

#Retrieve the record in dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a for loop 
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

#Print the column names list
print(columns)

#Print dataset

for data in dataset:
    print(data)

['Booking_Hour', 'n_Booking']
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


## Step 3:

Implement a stored procedure called GuestStatus.

Write a SQL CREATE PROCEDURE query for GuestStatus.

Combine the first and last name of each guest from the booking column using a CONCAT function.

Use the CASE function to implement the following order statuses for each guest:

If the Role in the Employee table is Manager or Assistant Manager then the status is Ready to pay.

If the Role in the Employee table is Head Chef then the status is Ready to serve.

If the Role in the Employee table is Assistant Chef then the status is Preparing order.

If the Role in the Employee table is Head Waiter then the status is Order served.

Perform a LEFT JOIN on the Bookings table with the Employees table ON EmployeeID.

Run the stored procedure query by invoking the execute module on the cursor object.

Invoke callproc to call the stored procedure.

Fetch the results in a variable called dataset.

Extract the names of the columns.

Print the names of the columns.

Print the sorted data using a for loop.

Close the connection to return it to the pool.

In [8]:
cursor.execute("DROP PROCEDURE IF EXISTS GuestStatus;")

stored_procedure_query = """
CREATE PROCEDURE GuestStatus()
BEGIN

SELECT
Bookings.BookingID AS OrderNumber,
CONCAT(GuestFirstName, '', GuestLastName) AS GuestFullName,

Role AS Employee,

CASE
WHEN Role IN ('Manager', 'Assistant Manager') THEN "Ready to pay"
WHEN Role = 'Head Chef' THEN "Ready to serve"
WHEN Role = 'Assistant Chef' THEN "Preparing order"
WHEN Role = 'Head Waiter' THEN "Order served"
ELSE "Pending"
END AS Status
FROM Bookings
LEFT JOIN Employees
ON Employees.EmployeeID = Bookings.EmployeeID;

END
"""
#Execute the query.
cursor.execute(stored_procedure_query)
#Call The procedure
cursor.callproc("GuestStatus")

#Retrieve the record in dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column names using list comprehension in a for loop 
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

#Print the column names list
print(columns)

#Print data
for data in dataset:
    print(data)

['OrderNumber', 'GuestFullName', 'Employee', 'Status']
(1, 'AnnaIversen', 'Manager', 'Ready to pay')
(2, 'JoakimIversen', 'Manager', 'Ready to pay')
(3, 'VanessaMcCarthy', 'Head Chef', 'Ready to serve')
(4, 'MarcosRomero', 'Assistant Chef', 'Preparing order')
(5, 'HirokiYamane', 'Assistant Manager', 'Ready to pay')
(6, 'DianaPinto', 'Head Waiter', 'Order served')
